# Image Coordinates and Captured Time

In [20]:
from PIL import Image
from PIL.ExifTags import TAGS
 
img = Image.open("LCMS-Range-Shuffled/train/00086.JPG")
print(img.info)
info = img._getexif()
if info is None:
    print("No Info")
else:
    for k, v in info.items():
        nice = TAGS.get(k, k)
        print( '%s (%s) = %s' % (nice, k, v) )


{'jfif': 257, 'jfif_version': (1, 1), 'dpi': (96, 96), 'jfif_unit': 1, 'jfif_density': (96, 96), 'exif': b"Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x01\x88%\x00\x04\x00\x00\x00\x01\x00\x00\x00\x1a\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x01\x00\x00\x00\x04\x02\x02\x00\x00\x00\x01\x00\x02\x00\x00\x00\x02S\x00\x00\x00\x00\x02\x00\x05\x00\x00\x00\x03\x00\x00\x00\xd4\x00\x03\x00\x02\x00\x00\x00\x02E\x00\x00\x00\x00\x04\x00\x05\x00\x00\x00\x03\x00\x00\x00\xec\x00\x05\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x06\x00\x05\x00\x00\x00\x01\x00\x00\x01\x04\x00\x07\x00\x05\x00\x00\x00\x03\x00\x00\x01\x0c\x00\x08\x00\x02\x00\x00\x00\x0310\x00\x00\x00\t\x00\x02\x00\x00\x00\x02V\x00\x00\x00\x00\x0b\x00\x05\x00\x00\x00\x01\x00\x00\x01$\x00\x0e\x00\x02\x00\x00\x00\x02T\x00\x00\x00\x00\x0f\x00\x05\x00\x00\x00\x01\x00\x00\x01,\x00\x12\x00\x02\x00\x00\x00\x06\x00\x00\x014\x00\x1d\x00\x02\x00\x00\x00\x0b\x00\x00\x01:\x00\x00\x00\x00\x00\x00\x00%\x00\x00\x00\x01\x00\x00\x005\x00\x00\x00\x01#\x04g\x86\x00\x98\

In [1]:
import exifread
from datetime import datetime

with open("LCMS-Range-Shuffled/train/00008.JPG", 'rb') as f:
    exif_dict = exifread.process_file(f)
    print(exif_dict.keys())
    print('capture time：', exif_dict['GPS GPSDate'])
    print('camera：', exif_dict['GPS GPSTimeStamp'])
    #print('camera type：', exif_dict['Image Model'])
    #print('image size：', exif_dict['EXIF ExifImageWidth'], exif_dict['EXIF ExifImageLength'])
    #date = exif_dict['GPS GPSDate']
    #time_str = date.strftime('%Y:%m:%d')
    #print(time_str)
    
    
    
    # Longitude
    lon_ref = exif_dict["GPS GPSLongitudeRef"].printable
    lon = exif_dict["GPS GPSLongitude"].printable[1:-1].replace(" ", "").replace("/", ",").split(",")
    lon = float(lon[0]) + float(lon[1]) / 60 + float(lon[2]) / float(lon[3]) / 3600
    if lon_ref != "E":
        lon = lon * (-1)

    # Latitude
    lat_ref = exif_dict["GPS GPSLatitudeRef"].printable
    lat = exif_dict["GPS GPSLatitude"].printable[1:-1].replace(" ", "").replace("/", ",").split(",")
    lat = float(lat[0]) + float(lat[1]) / 60 + float(lat[2]) / float(lat[3]) / 3600
    if lat_ref != "N":
        lat = lat * (-1)
    print('lat and long of image', (lat, lon))

    #00076,-37.89383776197222,144.6803922619722,2017-01-31

dict_keys(['GPS GPSVersionID', 'GPS GPSLatitudeRef', 'GPS GPSLatitude', 'GPS GPSLongitudeRef', 'GPS GPSLongitude', 'GPS GPSAltitudeRef', 'GPS GPSAltitude', 'GPS GPSTimeStamp', 'GPS GPSSatellites', 'GPS GPSStatus', 'GPS GPSDOP', 'GPS GPSTrackRef', 'GPS GPSTrack', 'GPS GPSMapDatum', 'GPS GPSDate', 'Image GPSInfo'])
capture time： 2017:01:31
camera： [13, 53, 5]
lat and long of image (-37.74830171075, 144.70169589380555)


In [63]:
import exifread, os
import csv

datasets = []
with open("train6000_Co_Dt.csv","w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Image","Latitude","Longitude","Date"])
    for filename in os.listdir("LCMS-Range-Shuffled/train/"):
        if filename.endswith('.JPG'):
            with open(os.path.join("LCMS-Range-Shuffled/train/", filename), "rb") as image: # Change "11.jpg" to filename variable
                exif = exifread.process_file(image)
                if "GPS GPSDate" in exif:
                    dt = str(exif['GPS GPSDate'])
                    date = dt.replace(":","-")
                    #print(dt.replace(":","-"))

                # Longitude
                lon_ref = exif["GPS GPSLongitudeRef"].printable
                lon = exif["GPS GPSLongitude"].printable[1:-1].replace(" ", "").replace("/", ",").split(",")
                lon = float(lon[0]) + float(lon[1]) / 60 + float(lon[2]) / float(lon[3]) / 3600
                if lon_ref != "E":
                    lon = lon * (-1)

                # Latitude
                lat_ref = exif["GPS GPSLatitudeRef"].printable
                lat = exif["GPS GPSLatitude"].printable[1:-1].replace(" ", "").replace("/", ",").split(",")
                lat = float(lat[0]) + float(lat[1]) / 60 + float(lat[2]) / float(lat[3]) / 3600
                if lat_ref != "N":
                    lat = lat * (-1)
                #print('lat and long of image', (lat, lon))

                datasets.append([str(filename)[:-4],str(lat),str(lon), date])
    
    writer.writerows(sorted(datasets))

In [37]:
["1123","2321",32]

['1123', '2321', 32]

[['00001', '-37.74838856772222', '144.70172799694444', '2017-01-31'],
 ['00002', '-37.747603951777776', '144.70138189202777', '2017-01-31'],
 ['00003', '-37.74707084972222', '144.7009730729722', '2017-01-31'],
 ['00004', '-37.747655468', '144.70147634938888', '2017-01-31'],
 ['00005', '-37.74847386355555', '144.70176865966667', '2017-01-31'],
 ['00006', '-37.74723577216667', '144.70106580563888', '2017-01-31'],
 ['00007', '-37.74803607669445', '144.7016968883611', '2017-01-31'],
 ['00008', '-37.74830171075', '144.70169589380555', '2017-01-31'],
 ['00009', '-37.747543267833336', '144.70129921463888', '2017-01-31'],
 ['00010', '-37.74747235430556', '144.70123067813887', '2017-01-31'],
 ['00011', '-37.74821313741667', '144.70167833863889', '2017-01-31'],
 ['00012', '-37.74715380786111', '144.70101811719442', '2017-01-31'],
 ['00013', '-37.74812422880556', '144.70167949358333', '2017-01-31'],
 ['00014', '-37.74864190561111', '144.70185736613888', '2017-01-31'],
 ['00015', '-37.747773204222

In [29]:
import exifread, os
from datetime import datetime

for filename in os.listdir("LCMS-Range-Shuffled/demo/"):
    if filename.endswith('.JPG'):
        with open(os.path.join(directoryInput, filename), "rb") as image: # Change "11.jpg" to filename variable
            exif = exifread.process_file(image)
            if "GPS GPSDate" in exif:
                dt = str(exif['GPS GPSDate'])
                print(dt.replace(":","-"))
                # into date and time
                dt_time = datetime.strptime(dt, '%Y:%m:%d')
                #year, month, day = dt.split(":", 2)
                #hour, minute, second = dtime.split(":", 2)
                print(dt_time)

2017-01-31
2017-01-31 00:00:00
2017-01-31
2017-01-31 00:00:00
2017-01-31
2017-01-31 00:00:00
2017-01-31
2017-01-31 00:00:00
